In [8]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

bond_data = pd.read_csv(r'C:\Users\Melanie\Desktop\optimization_engine\bond_data.csv')

BenchmarkWeight = 0.01 

bond_data['transaction_cost'] = bond_data['ask_price'] - bond_data['bid_price']
bond_data['daily_return'] = bond_data['expected_return']
bond_data['DTS'] = bond_data['duration'] * bond_data['OAS']

N = 100 
lambda_1 = 10
lambda_2 = 20

model = gp.Model("DynamicCorporateBondOptimization")

w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  
x = model.addVars(N, vtype=GRB.BINARY, name="x")  # whether bond i is chosen

model.setObjective(
    gp.quicksum((bond_data.loc[i, 'expected_return'] * w[i] - bond_data.loc[i, 'transaction_cost'] * x[i]) for i in range(N))  # return minus transaction cost
    - lambda_1 * gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N))  # OAS risk term
    - lambda_2 * gp.quicksum(bond_data.loc[i, 'DTS'] * w[i] for i in range(N)),  # DTS risk term
    GRB.MAXIMIZE
)

# # Risk constraints
# benchmark_OAS = 274.88 
# lower_bound = 0.9 * benchmark_OAS
# upper_bound = 1.1 * benchmark_OAS
# model.addConstr(gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N)) >= lower_bound, "MinOAS")
# model.addConstr(gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N)) <= upper_bound, "MaxOAS")

# # Liquidity
# Liquidity = gp.quicksum(bond_data.loc[i, 'liquidity_score'] * BenchmarkWeight for i in range(N))
# MinLiquidity = 0.9 * Liquidity
# model.addConstr(gp.quicksum(bond_data.loc[i, 'liquidity_score'] * w[i] for i in range(N)) >= MinLiquidity, "MinLiquidity")

# Transaction cost 
Benchmark_cost = gp.quicksum(bond_data.loc[i, 'transaction_cost'] * BenchmarkWeight for i in range(N))
lower_t_cost = 1 * Benchmark_cost
upper_t_cost = 1 * Benchmark_cost
model.addConstr(gp.quicksum(bond_data.loc[i, 'transaction_cost'] * x[i] for i in range(N)) >= lower_t_cost, "MintCost")
model.addConstr(gp.quicksum(bond_data.loc[i, 'transaction_cost'] * x[i] for i in range(N)) <= upper_t_cost, "MaxtCost")

# binary 
M = 100000
for i in range(N):
    model.addConstr(w[i] <= M * x[i], f"WeightSelection_{i}")

# sum of weights = 1
model.addConstr(gp.quicksum(w[i] for i in range(N)) == 1, "WeightSum")

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in range(N):
        print(f"Bond {bond_data.loc[i, 'ISIN']}: weight = {w[i].x:.4f}, chosen = {x[i].x:.0f}")
else:
    print("No optimal solution found.")

chosen_bonds = [bond_data.loc[i, 'ISIN'] for i in range(N) if x[i].x == 1]
if chosen_bonds:
    print("Chosen bonds:", chosen_bonds)
    #output weights of the chosen_bonds
    print("Weights of chosen bonds:", [w[i].x for i in range(N) if x[i].x == 1])
else:
    print("No bonds were chosen.")
    
#

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 103 rows, 200 columns and 500 nonzeros
Model fingerprint: 0x29555eeb
Variable types: 100 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+05]
  Objective range  [3e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 42 rows and 84 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


AttributeError: Unable to retrieve attribute 'x'

## 1. Repeatedly identifies an IIS and removes it until feasible 

* if the model is infeasible, the example repeatedly identifies an Irreducible Irreducible Inconsistent Subsystem (IIS) and removes one of the associated constraints from the model until the model becomes feasible. 
* It is sufficient to remove one constraint from the IIS to address that source of infeasibility, but that one IIS may not capture all sources of infeasibility. 
* therefore necessary to repeat the process until the model is feasible.

In [11]:
import sys

# Optimize 
status = model.Status
if status == GRB.UNBOUNDED:
    print("The model cannot be solved because it is unbounded")
    sys.exit(0)
if status == GRB.OPTIMAL:
    print(f"The optimal objective is {model.ObjVal:g}")
    sys.exit(0)
if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print(f"Optimization was stopped with status {status}")
    sys.exit(0)

# do IIS
print("The model is infeasible; computing IIS")
removed = []

# Loop until we reduce to a model that can be solved
while True:
    model.computeIIS()
    print("\nThe following constraint cannot be satisfied:")
    for c in model.getConstrs():
        if c.IISConstr:
            print(c.ConstrName)
            # Remove a single constraint from the model
            removed.append(str(c.ConstrName))
            model.remove(c)
            break
    print("")

    model.optimize()
    status = model.Status

    if status == GRB.UNBOUNDED:
        print("The model cannot be solved because it is unbounded")
        sys.exit(0)
    if status == GRB.OPTIMAL:
        break
    if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
        print(f"Optimization was stopped with status {status}")
        sys.exit(0)

print("\nThe following constraints were removed to get a feasible LP:")
print(removed)

The model is infeasible; computing IIS
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0       103         -         0       100         -           0s
        2         2         2         0         0         0           0s

IIS computed: 2 constraints, 0 bounds
IIS runtime: 0.05 seconds (0.01 work units)

The following constraint cannot be satisfied:
MintCost

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE

## 2. Allow contraints to be relaxed and add artificial variable 

* it allows the constraints of the model to be relaxed. 
* An artificial variable is added to each constraint. 
* The example sets the objective on the original variables to zero, and then solves a model that minimizes the total magnitude of the constraint relaxation.

In [13]:
# Optimize 
status = model.Status
if status == GRB.UNBOUNDED:
    print("The model cannot be solved because it is unbounded")
    sys.exit(0)
if status == GRB.OPTIMAL:
    print(f"The optimal objective is {model.ObjVal:g}")
    sys.exit(0)
if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print(f"Optimization was stopped with status {status}")
    sys.exit(0)

# Relax the constraints to make the model feasible
print("The model is infeasible; relaxing the constraints")
orignumvars = model.NumVars
model.feasRelaxS(0, False, False, True)
model.optimize()
status = model.Status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print(
        "The relaxed model cannot be solved \
           because it is infeasible or unbounded"
    )
    sys.exit(1)

if status != GRB.OPTIMAL:
    print(f"Optimization was stopped with status {status}")
    sys.exit(1)

print("\nSlack values:")
slacks = model.getVars()[orignumvars:]
for sv in slacks:
    if sv.X > 1e-6:
        print(f"{sv.VarName} = {sv.X:g}")

The model is infeasible; relaxing the constraints
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 103 rows, 304 columns and 604 nonzeros
Model fingerprint: 0x5b960e70
Variable types: 204 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 1e+00]
Found heuristic solution: objective 1.9486179
Presolve time: 0.00s
Presolved: 103 rows, 304 columns, 604 nonzeros
Variable types: 204 continuous

For constraint bounds, the slack variables start with "ArtP_" or "ArtN_" followed by the name of the original constraint. Starting with "ArtP_" means that the RHS (right-hand side) of the constraint needs to be decreased while "ArtN_" means that the RHS needs to be increased.

## 3. Feasopt 

In [ ]:
# This example adds artificial
# variables to each constraint, and then minimizes the sum of the
# artificial variables.  A solution with objective zero corresponds
# to a feasible solution to the input model.
#
# We can also use FeasRelax feature to do it. In this example, we
# use minrelax=1, i.e. optimizing the returned model finds a solution
# that minimizes the original objective, but only from among those
# solutions that minimize the sum of the artificial variables.

import sys
import gurobipy as gp

# create a copy to use FeasRelax feature later

feasmodel1 = model.copy()

# clear objective

model.setObjective(0.0)

# add slack variables

for c in model.getConstrs():
    sense = c.Sense
    if sense != ">":
        model.addVar(
            obj=1.0, name=f"ArtN_{c.ConstrName}", column=gp.Column([-1], [c])
        )
    if sense != "<":
        model.addVar(
            obj=1.0, name=f"ArtP_{c.ConstrName}", column=gp.Column([1], [c])
        )

# optimize modified model
model.optimize()
model.write("feasopt.lp")

# use FeasRelax feature
feasmodel1.feasRelaxS(0, True, False, True)
feasmodel1.write("feasopt1.lp")
feasmodel1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 103 rows, 304 columns and 604 nonzeros
Model fingerprint: 0xf5ac58d1
Variable types: 204 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.020000e+32
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1.02e+32 
No other solutions better than 0

Model is unbounded
Best objective 1.020000000000e+32, best bound -, gap -
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) 